# Prepare training data to compute the guess function

In [4]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import pandas as pd


## prepare all random configurations

## run all configurations (without guess) on a parametrized circuit

## convert execution result counts and configuration to a training row data

In [2]:
one_execution = { 'state_probability': 0, # [0.25, 1]
              'rx_theta': 0, # [0, 2*pi]
              'ry_theta': 0, # [0, 2*pi]
              'eta0': 0, # [1.0995574287564276, pi/2] -> only computed for 20 etas, and it should be the resulting entangled etas for eta0 [63º, 90º]
              'eta1': 0, # [0, pi/2] -> but this is a discrete variable only 20 values
              'bit0': 0, # [0 or 1] the left bit of the measured counts '00'
              'bit1': 0, # [0 or 1] the right bit of the measured counts '00'
              'eta_used': 0} # target value

state_probabilities = []
rx_thetas = []
ry_thetas = []
etas0 = []
etas1 = []
bits0 = []
bits1 = []
etas_used = []
data = { 'state_probability': state_probabilities,
              'rx_theta': rx_thetas,
              'ry_theta': ry_thetas,
              'eta0': etas0,
              'eta1': etas1,
              'bit0': bits0,
              'bit1': bits1,
              'eta_used': etas_used} 

## write training data to a CSV file

In [5]:
df = pd.DataFrame(data)
csv_file_name = "./entangled_executions.csv"
df.to_csv(csv_file_name)